<a href="https://colab.research.google.com/github/edcalderin/DeepLearning_SaturdaysAI/blob/master/Tareas/Tarea_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea: Recurrent Neural Networks
### Grupo: RNN
Integrantes:
<br>
* Erick David Calderin Morales
* Sharon Sarai Maygua Mendiola
* Rodrigo Antonio Aliaga Vélez

<br>

Indicaciones:
<rb>
* Debe realizar la siguiente tarea hasta el miercoles 16 de junio, 23:59 UTC - 4
* Debe hacer una copia de este notebook para poder editar el código.
* Debe poner el código faltante en las celdas que correspondan.
* Una vez finalizado el trabajo debe subir el link de su notebook (con permisos de lector) en la sección de "Tareas" del Módulo 3: Recurrent Neural Networks en Eduflow.

En la parte práctica de la clase, vimos cómo entrenar una red neuronal recurrente (RNN) para la tarea del análisis del sentimiento. Sin embargo, los resultados en el test set no fueron nada buenos. En este ejercicio, haremos unos cuantos cambios para crear un modelo que nos dé una precisión de más del 80%:



*   Cambiar el RNN por un **LSTM bidireccional**
*   Utilizar **pre-trained word embeddings** españoles
*   Regularización
*   Un optimizer distinto
*   Procesaremos solo los elementos que no son *padding* (**packed padded sequences**)

Para utilizar los embeddings, hay que bajarse el fichero de https://www.kaggle.com/rtatman/pretrained-word-vectors-for-spanish tal y como vimos en la parte práctica. 




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Bajarse el tokenizer español de SpaCy

!python -m spacy download es_core_news_sm

     |████████████████████████████████| 16.2MB 27.7MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp37-none-any.whl size=16172935 sha256=06435598f0b29466804406691b87ca55cc8d440eb4ef1b6fd984cd79e6f652b3
  Stored in directory: /tmp/pip-ephem-wheel-cache-j098rg5z/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


Recordad que después de bajarse el tokenizer, para que funcione correctamente hay que reanudar el runtime.

*Runtime -> Restart runtime*

# Dataset

Vamos a inizializar el dataset en torchtext tal y como hicimos en las prácticas. Sin embargo, vamos a añador un parámetro extra en *data.Field* de TEXT llamado *include_lengths=True*. Para poder procesar los elementos que no son padding, necesitamos saber la longitud de cada texto en el dataset.  

In [ ]:
# Inicializar torchtext dataset

import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy', tokenizer_language='es_core_news_sm', include_lengths=True)

SENTIMENT = data.LabelField(dtype = torch.float)

In [ ]:
# Indicar a torchtext qué campos corresponden a los distintos elementos del json

fields = {'texto': ('t', TEXT), 'sentimiento': ('s', SENTIMENT)}

In [ ]:
# Echamos un vistazo al corpus

import json
from pprint import pprint

PATH = 'drive/MyDrive/SaturdaysAI/data_sentimiento/'

data1 = []
with open(PATH + 'train.json') as file:
  for line in file:
    data1.append(json.loads(line))

pprint(data1[:5])
print(len(data1))

[{'sentimiento': 'neg',
  'texto': 'Llegamos a las 21:30, hicimos los pedidos, solo a las 23 vino la '
           'primera porción que pedimos, cuando reclamamos que faltaban '
           'porciones nos dijeran que ya se habían agotado (después de ya '
           '1:30hr de espera)! A las 24 cuando estábamos por desistir del '
           'restante de lo pedido, vino el mozo y garantizo que ya estaban '
           'listos! Al fin cuando llegaran no tenían gusto, sin sal, en el '
           'bobó de camarao, no había camarao y había pelotas de harina! La '
           'verdad muy decepcionante! Fuimos en un grupo de 15 personas, y '
           'nadie quedo satisfecho!'},
 {'sentimiento': 'neg',
  'texto': 'la camarera tardó mucho más de lo aceptable en atendernos,jamás '
           'trajo el pan,y no tenía ni idea quién había pedido cada cosa.\r\n'
           'la comida llegó a destiempo (unos comían pizza como entrada y yo '
           'una ensalada.. bueno,mi ensalada llegó una vez que 

Ahora vamos a leer el corpus. Vamos a utilizar los tres ficheros (**train.json**, **valid.json** y **test.json**) que vimos en la parte práctica. 

In [ ]:
# Leer corpus

PATH = 'drive/MyDrive/SaturdaysAI/data_sentimiento'

train_data, valid_data, test_data = data.TabularDataset.splits(
                                                            path = PATH, 
                                                            train = 'train.json', 
                                                            validation = 'valid.json', 
                                                            test = 'test.json',
                                                            format = 'json',
                                                            fields = fields
                                                            )

Como vamos a utilizar *pre-trained embeddings*, tenemos que añadirlos cuando construimos el vocabulario. Para inicializar el vocabulario en los pre-trained embeddings a 0, añadimos el parámetro *unk_init*.

In [ ]:
!unzip -u "drive/MyDrive/SaturdaysAI/words_espanioles.zip" -d "drive/MyDrive/SaturdaysAI/"

Archive:  drive/MyDrive/SaturdaysAI/words_espanioles.zip


In [ ]:
# Leer embeddings, este proceso puede tardar unos segundos

import torchtext.vocab as vocab

FILE_NAME = 'SBW-vectors-300-min5.txt'
PATH = 'drive/MyDrive/SaturdaysAI/'

spanish_embeddings = vocab.Vectors(FILE_NAME, cache=PATH)

In [ ]:
# Constuir vocabulario 

MAX_VOCAB_SIZE = 4000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, 
                 vectors = spanish_embeddings,
                 unk_init = torch.Tensor.normal_)

SENTIMENT.build_vocab(train_data)

In [ ]:
# Preparar train, valid y test iterators para entrenar el modelo

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
     (train_data, valid_data, test_data),
     batch_size = BATCH_SIZE,
     sort_within_batch = True,
     sort_key = lambda x: len(x.t),
     device = device
)

In [ ]:
# Para ver la dimensión de los pre-trained embeddings

pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([4002, 300])


# Modelo

Construimos el modelo que entrenaremos y evaluaremos. En esta tarea utilizamos un **LSTM bidireccional**. 

La *embedding* layer tendrá un parámetro extra, ***padding_idx=pad_idx***, que indica el índice del token *pad* para que el model no lo procese.

La *rnn* layer será ahora de tipo **nn.LSTM** con los parámetros siguientes:


*   **embedding_dim**: dimensión de los pre-trained embeddings
*   **hidden_dim**: dimensión de la hidden layer
*   **num_layer**: número de layers
*   **bidirectional**: queremos un LSTM bidireccional 
*   **dropout**: el dropout para regularizar la red neuronal

Luego añadimos una **layer linear** y *dropout*. La dimensión de la hidden layer que pasamos por la linear layer es el **doble** porque concatenamos las dos hidden layers con distintas direcciones. 

También vamos a definir el paso forward. Atención que utilizamos *dropout* y *packed_padded_sequence*. Al final concatenamos las dos hidden layers del Bidirectional LSTM. 



In [ ]:
# Construir modelo

import torch.nn as nn

class BiLSTM(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
    super().__init__()
    self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=pad_idx)

    self.rnn = nn.LSTM(input_size = embedding_dim, 
                       hidden_size = hidden_dim, 
                       num_layers = n_layers,
                       dropout = dropout, 
                       bidirectional = bidirectional
                       )
    self.fc = nn.Linear(2*hidden_dim, output_dim)
    self.dropout = nn.Dropout(dropout)

  # Forward pass
  def forward(self, text, text_lengths):

    embedded = self.dropout(self.embedding(text))

    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
    
    packed_output, (hidden, cell) = self.rnn(packed_embedded)

    output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

    hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))

    return self.fc(hidden)

In [ ]:
# Definimos parámetros, algunos (como hidden_dim o n_layers) los podéis cambiar y experimentar

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 450
OUTPUT_DIM = 1
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

# Inicializamos el modelo con todos los parámetros

model = BiLSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

In [ ]:
# Para ver el número de parámetros que entrenaremos en la red neuronal

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'El modelo tiene {count_parameters(model):,} parámetros')

El modelo tiene 13,643,101 parámetros


Finalmente, copiamos los pre-trained word embeddings y los metemos en la *embedding layer*. Luego reemplazamos los valores iniciales de la *embedding layer* con los pre-trained embeddings. 

In [ ]:
# Copiamos pre-trained embeddings

pretrained_embeddings = TEXT.vocab.vectors

# Inicializamos embedding layer

model.embedding.weight.data.copy_(TEXT.vocab.vectors)

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [-3.9439e-02, -4.2805e-02, -5.0221e-02,  ..., -2.0496e-02,
          2.3131e-02, -8.1015e-02],
        [ 4.4232e-02, -5.5097e-02,  4.4623e-02,  ..., -7.1947e-02,
          7.4214e-02, -2.5300e-02],
        [-6.0870e-02, -4.6083e-02,  9.3000e-05,  ..., -8.2717e-02,
          1.1736e-01, -4.8698e-02]])

# Entrenar 

Vamos a entrenar el modelo. Empezamos definiendo el *optimizer*. Esta vez vamos a utilizar **Adam**. Utilizamos la misma loss function que vimos en la parte práctica.

In [ ]:
# Optimizer

import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-3)

# loss function

criterion = nn.BCEWithLogitsLoss()

In [ ]:
# Modelo y loss function en GPU

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
# función para calcular accuracy 

def binary_accuracy(preds, y):
  rounded_preds = torch.round(torch.sigmoid(preds))
  correct = rounded_preds == y
  acc = correct.sum() / len(correct)
  return acc

Definimos la función para entrenar el modelo. Como que hemos incluido el parámetro *include_lengths=True*, en este caso *batch.t* es una tupla con el primer elemento un tensor de números y el segundo elemento la longitud de cada texto. Con lo cual, antes de pasar *batch.t*, tendremos que separar estos dos elementos:

**text, text_lengths = batch.t**

Y pasar las dos variables (*text* y *text_lengths*) al modelo. 

In [ ]:
def train(model, iterator, optimizer, criterion):

  epoch_loss = 0
  epoch_acc = 0

  model.train()

  for batch in iterator:
    
    optimizer.zero_grad()
    text, text_lengths = batch.t
    predictions = model(text, text_lengths).squeeze(1)
    loss = criterion(predictions, batch.s)
    acc = binary_accuracy(predictions, batch.s)
    loss.backward()
    optimizer.step()
    epoch_loss += loss.item()
    epoch_acc += acc.item()
  
  return epoch_loss / len(iterator), epoch_acc / len(iterator)

Para definir la función que evalúa el modelo, recordar que es muy similar a *train*, con alguna diferencia (ver ejercicio práctico). 

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    model.eval()

    for batch in iterator: 
        text, text_lengths = batch.t   
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.s)
        acc = binary_accuracy(predictions, batch.s)

        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
# Función para saber el tiempo que se tarda para entrenar cada epoch

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Vamos a entrenar tal y como lo hicimos en el ejercicio práctico. 

In [ ]:
# Entrenamos

N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'sent-modelo.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 0.694 | Train Acc: 47.24%
	 Val. Loss: 0.687 |  Val. Acc: 64.06%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 0.669 | Train Acc: 59.50%
	 Val. Loss: 0.648 |  Val. Acc: 52.86%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 0.355 | Train Acc: 85.94%
	 Val. Loss: 0.490 |  Val. Acc: 83.25%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 0.428 | Train Acc: 82.93%
	 Val. Loss: 0.431 |  Val. Acc: 82.90%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: 0.210 | Train Acc: 91.23%
	 Val. Loss: 0.280 |  Val. Acc: 91.32%
Epoch: 06 | Epoch Time: 0m 1s
	Train Loss: 0.121 | Train Acc: 95.43%
	 Val. Loss: 0.668 |  Val. Acc: 71.44%
Epoch: 07 | Epoch Time: 0m 1s
	Train Loss: 0.248 | Train Acc: 92.91%
	 Val. Loss: 0.837 |  Val. Acc: 71.18%
Epoch: 08 | Epoch Time: 0m 1s
	Train Loss: 0.289 | Train Acc: 91.35%
	 Val. Loss: 0.720 |  Val. Acc: 69.01%
Epoch: 09 | Epoch Time: 0m 1s
	Train Loss: 0.315 | Train Acc: 88.70%
	 Val. Loss: 0.694 |  Val. Acc: 67.62%
Epoch: 10 | Epoch Time: 0m 1

# Test

Vamos a ver la precisión del modelo en el test data. Deberías obtener una accuracy alrededor del 80%!

In [ ]:
# Resultados en el test set

MODEL_NAME = 'sent-modelo.pt'

model.load_state_dict(torch.load(MODEL_NAME))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.465 | Test Acc: 84.03%


Si quieres ver cómo funciona el modelo con tus propios comentarios positivos o negativos, podemos crear una función para hacer predicciones.  

In [ ]:
# Cargar el tokenizer de SpaCy

import spacy
nlp = spacy.load('es_core_news_sm')

# Función para predecir sentimiento 

def predict_sentiment(model, sentence):
    # modelo en modo evaluación
    model.eval()
    # tokenizar texto
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    # transformar palabras en sus índices del vocabulario
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    # convertir lista de índices en tensor
    tensor = torch.LongTensor(indexed).to(device)
    # añadir una dimensión para batch
    tensor = tensor.unsqueeze(1)
    # convertir length en un tensor
    length_tensor = torch.LongTensor(length)
    # predecir, utilizando sigmoid para obtener un número entre 0 y 1
    prediction = torch.sigmoid(model(tensor, length_tensor))
    if prediction.item() >= 0.5:
      return "negativo"
    else:
      return "positivo"

In [ ]:
# Podéis probar con el texto que queráis. 

TEXTO = "SI ENTENDIMOS ESTA TAREA"

predict_sentiment(model, TEXTO)

'positivo'

In [ ]:
TEXTOS=[
        'Mi trabajo es muy dificil',
        'Me aumentaron el suelo',
        'No me pagaron horas extras',
        'Me despidieron',
        'El pais lo gobierna un mal presidente',
        'Tengo covid',
        'Me ascendieron',
        'Tengo casa nueva'
]
for text in TEXTOS:
    print(text, '---> Sent:', predict_sentiment(model, text))

Mi trabajo es muy dificil ---> Sent: positivo
Me aumentaron el suelo ---> Sent: positivo
No me pagaron horas extras ---> Sent: negativo
Me despidieron ---> Sent: positivo
El pais lo gobierna un mal presidente ---> Sent: negativo
Tengo covid ---> Sent: positivo
Me ascendieron ---> Sent: positivo
Tengo casa nueva ---> Sent: positivo


Los falsos positivos en las predicciones son evidentes, aún falta por mejorar.